In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(True)
clear_output()
for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.Seasons[Game.Season]['Season']
    Game.take_domain_actions()
    Game.war_move()
    Game.clean_up()
    Game.clear_screen()

df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
print(set(df['Action']))
print()
print(set(df[df['Success?']==True]['Action']))
Game.War


0
Instructions for updating:
Use tf.cast instead.
Forgot to mark a capitalitiative 10
Forgot to mark a capital
Forgot to mark a capital


IndexError: single positional indexer is out-of-bounds

In [ ]:
Game.War.iloc[0]["Notes"]

In [ ]:
Game.errors

In [ ]:
Game.War

In [ ]:
Game.Troops

In [ ]:
Game.Provences[Game.Provences['Provence']=="Cliff's Lament"]

In [ ]:
from source.mapping import Mapping

In [ ]:
%matplotlib notebook

In [ ]:
maps = Mapping(Game)
maps.focus_domains(['Baruk-Azhik'])
maps.show(show_abbreviations=True, show_troops=True, line_len=90, borders=True, caravans=False, shipping=True)


In [ ]:
Game.Regents[Game.Regents['Regent']=='GG']

In [ ]:
Game.Projects

In [ ]:
self = Game
Regent = 'RA'
friend = 'MRn'

import numpy as np


temp = pd.merge(self.Provences[self.Provences['Regent']==friend][['Regent', 'Provence']].copy(), self.Troops.copy(), on='Provence', how='left').fillna(0)
temp = temp[temp['Type'] != 0]
temp = temp[temp['Regent_x'] != temp['Regent_y']]
temp = temp[['Provence', 'CR']].groupby('Provence').sum().reset_index()
# temp['roll'] = np.random.randint(1, 10,temp.shape[0])+temp['CR']
# temp = temp.sort_values('roll', ascending=False)
# Target = temp.iloc[0]['Provence']
# Target_CR = int(2*temp.iloc[0]['CR']/3)
# temp = self.Troops[self.Troops['Regent']=='Regent'].copy()
# temp = temp[temp['Provence'] != Target]
# temp = temp[temp['Garrisoned']==0]

temp

In [ ]:
self.Provences[self.Provences['Regent']==friend]

In [ ]:
temp = Game.Provences[Game.Provences['Regent'] == friend]

In [ ]:
temp

In [ ]:
allies, enemies = Game.allies_enemies(Regent)

In [ ]:
enemies.shape[0]

In [ ]:
 temp = pd.concat([self.Relationships[self.Relationships['Regent'] == Regent], self.Relationships[self.Relationships['Other']==Regent]], sort=False)

In [ ]:
Game.Provences[Game.Provences['Regent'] == friend]